In [7]:
from transformers import AutoTokenizer
import spacy
import re
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, ContinuousColorMapper, LinearColorMapper, ColorBar, LabelSet, Label
from bokeh.plotting import figure, show, output_notebook
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from gensim.models import KeyedVectors
from yellowbrick.cluster import KElbowVisualizer
from bs4 import BeautifulSoup
from sklearn.preprocessing import StandardScaler, normalize
import requests
import pandas as pd
from gensim.test.utils import common_texts
import gensim.downloader
import nltk
import seaborn as sns
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
from bokeh.palettes import Spectral10
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import string
import sys
from nltk.tokenize import word_tokenize, sent_tokenize
import ipywidgets as widgets
import umap
from sklearn.feature_extraction.text import CountVectorizer
import PyPDF2

- gegeven de distributie van woorden, wat als je alle worden weg filter welke in het middelpunt van de curve vallen (66%)

gegeven een lijst met keywords, hoe kunnen we topics uit de documenten halen welke van toepassing zijn op de keywords?

# Onderzoeksvraag 2

> *"Welke KPI's zitten verweven in de aangeleverde overheid documenten (denk aan geplande aantal huizen bijvoorbeeld), en wat zijn de overeenkomsten tussen deze KPI's? "*

Om antwoord te geven op het eerste deel van de vraag hebben wij handmatig een lijst met KPI's uit 3 documenten gehaald. We hebben deze documenten deels gelezen en handmatig een aantal selecties gemaakt van zin(en) waarin een KPI verweven zit. Hiermee kunnen we de unsupervised modellen verifieren en (mogelijk)sturen richting een gewenste uitkomst.

## KPI similaritie measarumnets

In [2]:
handmatige_kpi_collectie = pd.read_csv("handmatige_kpi_collectie.csv")
handmatige_kpi_collectie.head(3)

,KPI,Document,Bladzijde
0,In het centrum bouwen we ruim 10.000 woningen ...,Omgevingsvisie Breda 2040 Sterk en Veerkrachti...,9
1,We verbeteren de verbindingen voor voetgangers...,Omgevingsvisie Breda 2040 Sterk en Veerkrachti...,9
2,De noordelijke rondweg ondertunnelen we groten...,Omgevingsvisie Breda 2040 Sterk en Veerkrachti...,9


In [24]:
# Gebruikte documenten
set(handmatige_kpi_collectie.Document.values)

{'Nijmegen stad in beweging - ontwerp omgevingsvisie 2020-2040',
 'Omgevingsvisie Breda 2040 Sterk en Veerkrachtig - 01-07-2021',
 'Tilburg Omgevingsvisie 2040'}

In [4]:
handmatige_kpi_collectie.shape

(90, 3)

# Onderzoeksvraag 3

> *"Welke van de te testten modellen en/of Pipeline configuraties is het effectiefst in het extrapoleren van een vooraf samengestelde lijst met KPI's? "*

<img src="" alt="Voeg hiero afbeelding van pipeline schets toe" />
*Voeg hiero afbeelding van pipeline schets toe*


Om KPI extrapolatie zo goed mogelijk op te stellen gaan wij gegeven overheids documenten opdelen in stukken tekst van N zinnen, waarbij N variable getest kan worden. Vervolgens worden al onze KPI's vergeleken met de stukken tekst. Een stuk tekst moet meer dan X overheenkomen met in ieder geval 1 KPI voordat hij wordt doorgelaten naar de rest van de pipeline. Hiervoor dient eerst getest te worden welke van de 2 vergelijkings methodes (semantisch en syntactisch) het beste resultaat geeft. 

## Overheenkomst meting KPI en delen van overheids documenten
Eerst defineren wij de methodes om semantisch en syntactische overheenkomste te meten. Daarna bekijken wij de top 5 meest overheenkomende stukken tekst per methode. Hieruit kunnen wij bepalen welke van de 2 aanpakken het beste werkt.

In [5]:
import spacy
import warnings
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import wordpunct_tokenize
from tqdm import tqdm
from difflib import SequenceMatcher

lemmatizer = WordNetLemmatizer()
spacy_model = spacy.load("nl_core_news_lg")

def preprocess_phrase(phrase):
    split_phrase = wordpunct_tokenize(phrase.lower())
    return " ".join([ lemmatizer.lemmatize(i) for i in split_phrase ])

def semantic_similarity(X, Y, max_out=None):
    # This function loads the large English language model in spaCy
    # and uses it to calculate semantic similarity between each phrase in Y and the input phrase X.
    # Note that this function assumes that the phrases in Y are in English, 
    # and it may not work as well for other languages or for very short or ungrammatical phrases.
    
    # Create a Doc object for X
    doc1 = spacy_model(preprocess_phrase(X))

    # Calculate semantic similarity for each phrase in Y
    similarities = []
    for phrase in Y:
        doc2 = spacy_model(preprocess_phrase(phrase))
        similarity = doc1.similarity(doc2)
        similarities.append((doc2, similarity))
        
    # return similarities
    if not max_out:
        max_out = len(similarities)
    return sorted(similarities, key=lambda a: a[1], reverse=True)[0:max_out]

def syntactic_similarity(X, Y, max_out=None):
    # This function takes in two parameters: 
    # X, the single phrase to compare against the list of phrases
    # Y, the list of phrases to compare against. 
    # It returns a tuple containing the most similar phrase from Y and the corresponding syntactic similarity score.
    match_ratios = []
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for phrase in Y:
            matcher = SequenceMatcher(None, preprocess_phrase(X), preprocess_phrase(phrase))
            ratio = matcher.ratio()
            match_ratios.append((phrase, ratio))
    
    if not max_out:
        max_out = len(match_ratios)
        
    return sorted(match_ratios, key=lambda a: a[1], reverse=True)[0:max_out]

D:\Users\luc\anaconda3\envs\sem4_base\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'nl_core_news_lg' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
from tqdm import tqdm
import PyPDF2

# get file content
page_content = []
with open("./Definitief concept Omgevingsvisie Weert (20-08-2021).pdf", "rb") as file:
    pdf_reader = PyPDF2.PdfFileReader(file)
    pages = []
    print("Getting page content")
    for page_number in tqdm(range(pdf_reader.getNumPages())):
        page = pdf_reader.getPage(page_number)
        page_content.append(page.extract_text())
        
# read KPI collection
kpis = list(pd.read_csv("handmatige_kpi_collectie.csv").KPI.astype(str))

100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 45.05it/s]


In [9]:
phrases = re.split("\.|\n", "\n".join(page_content))
chuncks = []
N_chunks = 3
while len(phrases) >= N_chunks:
    chunk = " ".join(phrases[0:N_chunks])
    chuncks.append(chunk)
    phrases = phrases[1:]

In [10]:
chunk_synt_syms = dict()
for kpi in tqdm(kpis):
    chunk_synt_syms[kpi] = syntactic_similarity(kpi, chuncks, 5)

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [03:01<00:00,  2.01s/it]


In [11]:
for i in chunk_synt_syms:
    print(i)
    for sim_chunck in chunk_synt_syms[i]:
        print(sim_chunck)
    print("----------------------------")

In het centrum bouwen we ruim 10.000 woningen en zorgen we ervoor dat hier alles goed bereikbaar is.
('gemiddelde woningdichtheid in het bebouwde gebied van Weert is 15 woningen per ha  Dit  maakt Weert als geheel suburbaan', 0.4636363636363636)
('  Een centrumstedelijk woonmilieu (> 70 woningen per ha) komt in Weert niet voor  De ', 0.4540540540540541)
('  Over het algemeen zijn deze woningen slecht geïsoleerd  Hier ligt een enorme ', 0.449438202247191)
('Over het algemeen zijn deze woningen slecht geïsoleerd  Hier ligt een enorme  verduurzamingsopgave', 0.44221105527638194)
('  Tot 2040 zullen daar zo’n 62 000 woningen moeten worden gebouwd', 0.42857142857142855)
----------------------------
We verbeteren de verbindingen voor voetgangers, over het water en via het spoor. We leggen mobiliteitshubs en een Singelpark aan.
('Belgische overheid bestaat de wens om deze verbinding geschikt te maken voor  personenvervoer  Niet alleen om Weert en Hamont met elkaar te verbinden, maar ook om ',

In [12]:
chunk_sema_syms = dict()
for kpi in tqdm(kpis):
    chunk_sema_syms[kpi] = semantic_similarity(kpi, chuncks, 5)

  0%|                                                                                           | 0/90 [00:00<?, ?it/s]C:\Users\luc\AppData\Local\Temp\ipykernel_23448\3250075912.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = doc1.similarity(doc2)
100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [20:20<00:00, 13.56s/it]


In [13]:
for i in chunk_sema_syms:
    print(i)
    for sim_chunck in chunk_sema_syms[i]:
        print(sim_chunck)
    print("----------------------------")

In het centrum bouwen we ruim 10.000 woningen en zorgen we ervoor dat hier alles goed bereikbaar is.
(we moeten ons hier constant bewust van zijn en we moeten ons snel kunnen aanpassen aan onze omgeving integrale veiligheid is het concept om complexe , meerduidige, 0.8628122940513293)
(2030 de groene thuishaven waar het goed wonen , werken , leven , verblijven is en waarin iedereen meetelt en meedoet om dit voor elkaar te krijgen moeten we nu al belangrijke keuzes, 0.8528044982228912)
(gewenste ontwikkelingen weer van de omgeving waarin we wonen , werken en onze vrije tijd doorbrengen en daarnaast onze kwaliteiten die we koesteren en willen behouden weert is in, 0.840110418360045)
(we moeten ons hier constant bewust van zijn en we moeten ons snel kunnen aanpassen aan onze omgeving, 0.8397916812786655)
(weert is goed bereikbaar en er zijn geen grote knelpunten we hebben een intercitystation en eindhoven airport en maastricht - aachen airport liggen op korte afstand en verbinden ons, 0.8

### Conclusie
Na aanleiding van bovenstaande uitkomsten kunnen wij concluderen dat het gebruik van semantische overheenkomst meting beter werkt dan syntactische metingen.

Zoals te zien is in de syntactische overheenkomst metingen zijn de scores aanzienlijk laag, als wij een strenge drempel waarde willen toepassen op hoever een stuk tekst overheen moet komen met een van onze KPI's komen geen enkele stukken tekst door onze filter aanpak heen. Daarnaast zien wij dat de inhoud van de syntactisch overheenkomende zinnen vrijwel geen KPI's bevatten.

Daarnaast zien wij het tegenovergestelde in de semantische overheenkomst meting. In vrijwel alle stukken tekst is terug te zien dat het gaat om iets meetbaars en daarnaast zijn de scores hoog genoeg om een strenge drempel waarde op toe te passen.

---

## Pipeline testing

Wij testen de pipeline op *Definitief concept Omgevingsvisie Weert (20-08-2021)* om aan te tonen dat de Pipeline functioneerd naar behoren.

### Document to chuncks & Semantic Similarity Measure

Onderstaande code is gebruikt om de eerder verwerkte stukken tekst van het test document semantisch te vergelijken. De uitkomst van deze vergelijking is opgeslagen voor toekomstige referenties.
```python
# store the chuncks which are sementicly similair to a given KPI
chunk_sema_syms = dict()

for kpi in tqdm(kpis):
    sema_chuncks = semantic_similarity(kpi, chuncks)
    
    # only store if the chunk and the KPI are 90% similair or more
    kpi_sema_treshhold = 0.8
    chunk_sema_syms[kpi] = [ i for i in sema_chuncks if i[1] >= kpi_sema_treshhold ]
    
parsed_chuncks = [ str(j[0]) for i in list(chunk_sema_syms.values()) for j in i if i ]
json_object = json.dumps(parsed_chuncks, indent=4)

with open("parsed_chuncks.json", "w") as file:
    file.write(json_object)
```

```stdout
100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [18:28<00:00, 12.32s/it]
```

In [16]:
import json

parsed_chuncks = None

with open("parsed_chuncks.json", "r") as file:
    parsed_chuncks = json.load(file)

In [17]:
parsed_chuncks

['doorbrengen en daarnaast onze kwaliteiten die we koesteren en willen behouden weert is in 2030 de groene thuishaven waar het goed wonen , werken , leven , verblijven is en waarin iedereen meetelt en meedoet om dit voor elkaar te krijgen moeten we nu al belangrijke keuzes',
 'weert is in 2030 de groene thuishaven waar het goed wonen , werken , leven , verblijven is en waarin iedereen meetelt en meedoet om dit voor elkaar te krijgen moeten we nu al belangrijke keuzes maken',
 'we moeten ons hier constant bewust van zijn en we moeten ons snel kunnen aanpassen aan onze omgeving integrale veiligheid is het concept om complexe , meerduidige veiligheidsvraagstukken op een brede , integrale manier te benaderen hiervoor is',
 'maatschappelijke norman verschuiven en de rechtsstaat staat onder druk we moeten ons hier constant bewust van zijn en we moeten ons snel kunnen aanpassen aan onze omgeving integrale veiligheid is het concept om complexe , meerduidige',
 'we moeten ons hier constant bewu

---

### Pipeline excecution

Voor de laatste stap in onze pipeline dienen wij eerst nog een custom evaluatie transformer te schrijven.

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
import umap
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, ContinuousColorMapper, LinearColorMapper, ColorBar, LabelSet, Label
from bokeh.palettes import Spectral10
%matplotlib inline


sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})
output_notebook()
    

class UmapEvaluationTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
        self.encoder = LabelEncoder()
        self.reducer = umap.UMAP()

    def fit(self, X, y=None):
        self.scaler.fit(X)
        self.reducer.fit(X)
        return self

    def transform(self, X, y=None):        
        # Apply UMAP to scaled topic matrix
        scaled_topic_data = self.scaler.transform(X)
        embedding = self.reducer.transform(scaled_topic_data)
        embeddingdf = pd.DataFrame(embedding, columns=("x", "y"))

        # Encode topics to numeric values
        highest_scoring_topics = list(np.argmax(X, axis=1).astype(str))
        self.encoder.fit(highest_scoring_topics)
        embeddingdf["highest_scoring_topic"] = self.encoder.transform(highest_scoring_topics)

        # Get mapping data source
        datasource = ColumnDataSource(embeddingdf)
        factors = list(set(embeddingdf["highest_scoring_topic"].values))
        color_mapping = LinearColorMapper(
            palette='Viridis256',
            low=embeddingdf["highest_scoring_topic"].min(),
            high=embeddingdf["highest_scoring_topic"].max()
        )

        # Create interactive Bokeh figure
        plot_figure = figure(
            title='UMAP projection of Topic Modeling Outcome',
            width=1000,
            height=800,
            tools=('pan, wheel_zoom, reset')
        )

        # Plot data points
        plot_figure.circle(
            'x',
            'y',
            source=datasource,
            color=dict(field='highest_scoring_topic', transform=color_mapping),
            size=4
        )

        # TODO: make keywords apear on hover
        # Add on-hover functionality to data points
        plot_figure.add_tools(HoverTool(tooltips="""
        <div>
            <div style="border: 1px black;">
                <span style='font-size: 12px'>Topic: @highest_scoring_topic</span>
                <span style='font-size: 12px'>, Chunck keywords go here</span>
            </div>
        </div>
        """))

        # Add color bar
        color_bar = ColorBar(
            color_mapper = color_mapping,
            title = 'Highest scoring topic'
        )
        plot_figure.add_layout(color_bar, 'right')

        # Show plot
        show(plot_figure)
        
        return X

Loading BokehJS ...

Testen of een andere methode van scoring (als LDA TFIDF gebruikt testen wij count en TFIDF) om te kijken of dit tot ruis/valse conclusies kan lijden

What is topic coherence? Topic Coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference.

- zelf in groepje van 3 (chuncks), evalueer welke chuncks het beste resultaat geeft
- per topic, hoogst scorende X = 100 woorden laten zien
- TF-IDF op input, deze woorden komen in de uitkomst voor, score de model o.b.v. hoe veel deze overheen komen
- keyword analyses

=====
Week 4 testen:

Zelf een verwachtings lijst met keywords voor topics opstellen en kijken of dit in uitkomst terug komt. Dit is de uitkomst van summerization. 

In [22]:
from sklearn.decomposition import LatentDirichletAllocation
from umap import UMAP
from sklearn.pipeline import Pipeline


vec = TfidfVectorizer(tokenizer=word_tokenize)
lda = LatentDirichletAllocation(n_components=10, random_state=0)
umap_evaluation = UmapEvaluationTransformer()

pipe = Pipeline([
    ("vec", vec),
    ("model", lda),
    ("umap_eval", umap_evaluation)
])

topic_matrix = pipe.fit_transform(parsed_chuncks)

D:\Users\luc\anaconda3\envs\sem4_base\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
D:\Users\luc\anaconda3\envs\sem4_base\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
